In [1]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
import keras
model = tf.keras.models.load_model("Digit_Recognizer")
from ipynb.fs.full.algo_sudoku_solver import solve

Using TensorFlow backend.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
# perspective wrapping

def crop_and_warp(img, crop_rect):

    top_left, top_right, bottom_right, bottom_left = crop_rect[0], crop_rect[1], crop_rect[2], crop_rect[3]
    
    src = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    
    side = max([
        distance_between(bottom_right, top_right),
        distance_between(top_left, bottom_left),
        distance_between(bottom_right, bottom_left),
        distance_between(top_left, top_right)
    ])
    
    dst = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype='float32')

    m = cv2.getPerspectiveTransform(src, dst)
    return cv2.warpPerspective(img, m, (int(side), int(side)))

In [3]:
def distance_between(p1, p2):
    a = p2[0] - p1[0]
    b = p2[1] - p1[1]
    return np.sqrt((a ** 2) + (b ** 2))

In [4]:
# finding corners of biggest contour
import operator
def find_corners_of_largest_polygon(cnt):

    polygon = cnt  

    bottom_right, _ = max(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    top_left, _ = min(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    bottom_left, _ = min(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
    top_right, _ = max(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))

    return [polygon[top_left][0], polygon[top_right][0], polygon[bottom_right][0], polygon[bottom_left][0]]


In [7]:
# to draw 81 squares on image
def infer_grid_lining(img):

    squares = []
    side = img.shape[:1]
    side = side[0] / 9
    for i in range(9):
        for j in range(9):
            p1 = (i * side, j * side)  
            p2 = ((i + 1) * side, (j + 1) * side)
            squares.append((p1, p2))
    return squares

In [8]:
def cut_from_rect(img, rect):
    return img[int(rect[0][1]):int(rect[1][1]), int(rect[0][0]):int(rect[1][0])]


def scale_and_centre(img, size, margin=0, background=0):
    h, w = img.shape[:2]

    def centre_pad(length):
        if length % 2 == 0:
            side1 = int((size - length) / 2)
            side2 = side1
        else:
            side1 = int((size - length) / 2)
            side2 = side1 + 1
        return side1, side2

    def scale(r, x):
        return int(r * x)

    if h > w:
        t_pad = int(margin / 2)
        b_pad = t_pad
        ratio = (size - margin) / h
        w, h = scale(ratio, w), scale(ratio, h)
        l_pad, r_pad = centre_pad(w)
    else:
        l_pad = int(margin / 2)
        r_pad = l_pad
        ratio = (size - margin) / w
        w, h = scale(ratio, w), scale(ratio, h)
        t_pad, b_pad = centre_pad(h)

    img = cv2.resize(img, (w, h))
    img = cv2.copyMakeBorder(img, t_pad, b_pad, l_pad, r_pad, cv2.BORDER_CONSTANT, None, background)
    return cv2.resize(img, (size, size))


def find_largest_feature(inp_img, scan_tl=None, scan_br=None):

    img = inp_img.copy()
    height, width = img.shape[:2]

    max_area = 0
    seed_point = (None, None)

    if scan_tl is None:
        scan_tl = [0, 0]

    if scan_br is None:
        scan_br = [width, height]

    for x in range(scan_tl[0], scan_br[0]):
        for y in range(scan_tl[1], scan_br[1]):
            if img.item(y, x) == 255 and x < width and y < height:
                area = cv2.floodFill(img, None, (x, y), 64)
            if area[0] > max_area: 
                    max_area = area[0]
                    seed_point = (x, y)


    for x in range(width):
        for y in range(height):
            if img.item(y, x) == 255 and x < width and y < height:
                cv2.floodFill(img, None, (x, y), 64)

    mask = np.zeros((height + 2, width + 2), np.uint8) 

    if all([p is not None for p in seed_point]):
        cv2.floodFill(img, mask, seed_point, 255)

    top, bottom, left, right = height, 0, width, 0

    for x in range(width):
        for y in range(height):
            if img.item(y, x) == 64:
                cv2.floodFill(img, mask, (x, y), 0)

            if img.item(y, x) == 255:
                top = y if y < top else top
                bottom = y if y > bottom else bottom
                left = x if x < left else left
                right = x if x > right else right

    bbox = [[left, top], [right, bottom]]
    return img, np.array(bbox, dtype='float32'), seed_point


def extract_digit(img, rect, size):

    digit = cut_from_rect(img, rect)

    h, w = digit.shape[:2]
    margin = int(np.mean([h, w]) / 2.5)
    _, bbox, seed = find_largest_feature(digit, [margin, margin], [w - margin, h - margin])
    digit = cut_from_rect(digit, bbox)

    w = bbox[1][0] - bbox[0][0]
    h = bbox[1][1] - bbox[0][1]

    if w > 0 and h > 0 and (w * h) > 100 and len(digit) > 0:
        return scale_and_centre(digit, size, 4)
    else:
        return np.zeros((size, size), np.uint8)

    
def pre_process_image(img, skip_dilate=False):

    proc = cv2.GaussianBlur(img.copy(), (9, 9), 0)
    proc = cv2.adaptiveThreshold(proc, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    proc = cv2.bitwise_not(proc, proc)

    if not skip_dilate:
        kernel = np.array([[0., 1., 0.], [1., 1., 1.], [0., 1., 0.]])
        proc = cv2.dilate(proc, kernel)

    return proc

def get_digits(img, squares, size):
    digits = []
    img = pre_process_image(img.copy(), skip_dilate=True)
    for square in squares:
        digits.append(extract_digit(img, square, size))
    return digits

In [9]:
def show_digits(digits, colour=255):
    rows = []
    with_border = [cv2.copyMakeBorder(img.copy(), 1, 1, 1, 1, cv2.BORDER_CONSTANT, None, colour) for img in digits]
    for i in range(9):
        row = np.concatenate(with_border[i * 9:((i + 1) * 9)], axis=0)
        rows.append(row)
    return(np.concatenate(rows,axis=1))

In [10]:
def print_output(sudoku, solved_sudoku, img):
    
    side = img.shape[:1] 
    length= int(side[0])  
    side = int(side[0]/9)
    margin = int((side/4))
    text_size = int(side/25)
    
    if text_size <1:
        text_size = 0.6
    
    font = cv2.FONT_HERSHEY_SIMPLEX

    for i in range(9):
        for j in range(9):
            
            if(sudoku[i,j]==0):
                text = chr(65+i)+ str(j+1)
                text= solved_sudoku[text]
                img = cv2.putText(img, text, (j*side +5 ,i*side+ 25), font, text_size , (0,225,0), 2 , cv2.LINE_AA) 
    return img

In [11]:
def unwrap(frame ,img ,coordinates):
    
    rows,cols,ch = img.shape
    row,col,ch2 = frame.shape

    pts1 = np.float32([[0,0],[cols,0],[cols,rows]])
    pts2 = np.float32([coordinates[0],coordinates[1],coordinates[2]])

    M = cv2.getAffineTransform(pts1,pts2)
    dst = cv2.warpAffine(img,M,(col,row))
    
    return dst

In [13]:
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH,900)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,600)

global_solved_sudoku = False
flag=0

while(True):
    
    ret, frame = cap.read()
    img = frame.copy()
    canvas = np.zeros(img.shape, np.uint8)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(img, (3,3) ,0 )
    thresh =cv2.adaptiveThreshold(blur, 255 , cv2.ADAPTIVE_THRESH_MEAN_C,  cv2.THRESH_BINARY_INV, 11, 2)
     
    image,contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    contours_sizes= [(cv2.contourArea(cnt), cnt) for cnt in contours]
    biggest_contour = max(contours_sizes, key=lambda x: x[0])[1]

    cnt = biggest_contour
    contour_area = cv2.contourArea(cnt)
    
    if contour_area > 40000.0:
    
        perimeter = cv2.arcLength(cnt,True)
        epsilon = 0.01*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)

        cv2.drawContours(canvas, cnt, -1, (255, 255, 255), -1)
        cv2.drawContours(canvas, [approx],-1, (255, 255, 255), -1)

        extLeft = tuple(cnt[cnt[:, :, 0].argmin()][0])
        extRight = tuple(cnt[cnt[:, :, 0].argmax()][0])
        extTop = tuple(cnt[cnt[:, :, 1].argmin()][0])
        extBot = tuple(cnt[cnt[:, :, 1].argmax()][0])

        coordinates =  find_corners_of_largest_polygon(cnt)
        crop_image = crop_and_warp(thresh, coordinates)
        actual_crop_image = crop_and_warp(frame, coordinates)
        crop_image = cv2.medianBlur(crop_image, 3)

        crop_image = cv2.bitwise_not(crop_image)
        squares = infer_grid_lining(crop_image)

        extracted = get_digits(crop_image, squares, 28)
        crop_image = show_digits(extracted)

        pics = infer_grid(crop_image)
        
        sudoku_arr =[]

        for i in range(len(pics)):
            pic= pics[i]
            pic = cv2.resize(pic,(28,28))
            pic = pic.reshape((1,28,28,1))
            pixel_sum = np.sum(pic)

            predicted_digit=0

            if pixel_sum < 4000.0:
                predicted_digit = 0
            else:

                prediction = model.predict(cv2.bitwise_not(pic))
                predicted_digit = prediction.argmax()

            sudoku_arr.append(predicted_digit)

        sudoku_str=""
        for digit in sudoku_arr:
            if(digit==0):
                sudoku_str+= "."
            else:    
                sudoku_str+= str(digit)

        solved_sudoku = solve(sudoku_str)
        sudoku_arr = np.array(sudoku_arr)
        sudoku_arr = sudoku_arr.reshape(9,9) 
        
        if type(False) != type(solved_sudoku):

            global_solved_sudoku = solved_sudoku
            flag=1
            actual_crop_image = print_output(sudoku_arr, solved_sudoku, actual_crop_image)

            unwarped_img = unwrap(frame, actual_crop_image, coordinates)

            r,c,ch = unwarped_img.shape
            canvas = cv2.resize(canvas,(c,r))

            unwarped_img = unwarped_img.astype(float)
            frame = frame.astype(float)
            canvas = canvas.astype(float)/255

            unwarped_img = cv2.multiply(canvas, unwarped_img)
            frame = cv2.multiply(1.0 - canvas, frame)
            
            frame = cv2.add(unwarped_img, frame)
            frame = frame/255
            cv2.imshow("Sudoku Solver", frame)

        elif type(False) == type(solved_sudoku) and flag==1:
            
            actual_crop_image = print_output(sudoku_arr, global_solved_sudoku, actual_crop_image)
            
            unwarped_img = unwrap(frame, actual_crop_image, coordinates)

            r,c,ch = unwarped_img.shape
            canvas = cv2.resize(canvas,(c,r))

            unwarped_img = unwarped_img.astype(float)
            frame = frame.astype(float)
            canvas = canvas.astype(float)/255

            unwarped_img = cv2.multiply(canvas, unwarped_img)
            frame = cv2.multiply(1.0 - canvas, frame)
            
            frame = cv2.add(unwarped_img, frame)
            frame = frame/255
            cv2.imshow("Sudoku Solver", frame)
        
    cv2.imshow("Sudoku Solver", frame)
    
    if(cv2.waitKey(40)==27):
        break

cv2.destroyAllWindows()